# DiploDatos 2021


### Categorización de publicaciones de productos realizadas en Mercado Libre

### 03 - Introducción al Aprendizaje Automático

#### Condiciones generales que aplican a todos los prácticos:
   - Las notebooks tienen que ser 100% reproducibles, es decir al ejecutar las celdas tal cuál como se entrega la notebook se deben obtener los mismos resultados sin errores.
   - Código legible, haciendo buen uso de las celdas de la notebook y en lo posible seguir estándares de código para Python (https://www.python.org/dev/peps/pep-0008/).
   - Utilizar celdas tipo "Markdown" para ir guiando el análisis.
   - Limpiar el output de las celdas antes de entregar el notebook (ir a Kernel --> Restart Kernel and Clear All Ouputs).
   - Incluir conclusiones del análisis que se hizo en la sección "Conclusiones". Tratar de aportar valor en esta sección, ser creativo! 

## 1. Consignas

Opcional: entregar la solución en scripts de Python. Se lo puede separar al proyecto en *data.py*, *models.py*, *metrics.py*, *train.py*, etc

#### Sección 0: Splitteo en train / test set

- Splittear en entrenamiento (80%) y test (20%).

#### Sección 1:

Tomamos lo generado en el TP anterior, por lo tanto partir de la ya hecho para tener:

- Sequences

- Labels

- Capa de Embedding

(Tener en cuenta que la tokenización se hace por separado en los conjuntos de entrenamiento y test, para evitar leakage de datos).

#### Sección 2: Splitteo en train / validation set

- Splittear al conjunto de entrenamiento en train y validation utilizando el método de Stratified K-Folds cross validation:
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold

#### Sección 3: Entrenamiento de un modelo

- Relizar el entrenamiento de un modelo.

- Se aconseja comenzar por una red neuronal feed foward, y/o capas del tipo *LSTM*.

- La primera capa de la red será la *embedding_layer* que ya hayan definido y la última capa será una capa densa con cantidad de neuronas = cantidad de clases y función de activación *softmax*.

- Entrenar modelos que utilicen tanto los embeddings pre-entrenados como los customs.

- Experimentar utilizando dropout y batch normalization.

- Utilizar *checkpoints* para guardar el modelo.

- Utilizar como *loss* la *'sparse_categorical_crossentropy'*, y se recomienda utilizar *Adam* como *optimizer*.

#### Sección 4: Evaluación del modelo

- Gráficar curvas de loss y accuracy para train y validation set (se puede hacer utilizando el model history de keras: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History).

- Utilizar balance accuracy para medir la performance del modelo en validaiton y test: 
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html

- Diferenciar el score según *label_quality*. Es decir, calcular el score para el set de validation y test separando según *reliable* y *unreliable*.

#### Sección 5: Análisis de resultados

- Realizar análisis de que modelo performó mejor, generar un csv con los resultados en validation y test de cada modelo.

#### Sección 6: Exportado de predicciones

- Generar un dataframe con las predicciones del conjunto de test del mejor modelo y exportarlo a un *csv*.

## 2. Código y análisis

Imports necesarios

In [ ]:
import pandas as pd

Lectura de dataset reducido

In [ ]:
df_dataset = pd.read_csv('dataset.csv')

In [ ]:
# TO DO

## 3. Conclusiones

In [ ]:
# TO DO